In [ ]:
## Importing standards libraries

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
#import torch

## Importing libraries for the Gaussian Mixture Model

from scipy.stats import multivariate_normal
from scipy.stats import norm
import seaborn as sns

## Convergence metrics

from scipy.stats import wasserstein_distance
from scipy.stats import entropy # KL-divergence


## Import functions for the experiment : 

from experiment_functions import *
from IPLA_Exp_Functions import *
from IPLA_functions_MMLE_V2 import *

### MMLE Experiment with Banana Distribution

In this example, we will work on $ \mathbb{R}^{dx} $ Let us assume the following “banana” prior:

$ p(x) \propto \exp\left(-\frac{x_1^2}{10} - \frac{x_2^2}{10} - 2(x_2 - x_1^2)^2\right), $

where $ x \in \mathbb{R}^{dx} $ We would like to then use the following likelihood:

$ p(y|x) = \mathcal{N}(y; Hx, \sigma^2 I), $

where $ H \in \mathbb{R}^{dx} $ and $ y \in \mathbb{R} $ 

Our first goal is to sample from $ p(x|y) $ efficiently given the parameter $H$. Then, we will proceed to estimate the Maximum Marginal Likelihood of the model, given the observation. 


In [ ]:
def grad_banana(x):

    x = -x/5

    x[:, 0] += 8 * x[:, 0] * (x[:, 1] - x[:, 0]**2)

    x[:, 1] -= 4 * (x[:, 1] - x[:, 0]**2)

    return x


In [ ]:
sample_init = np.random.randn(10000, 2)

plot_sample_dx(grad_banana(sample_init))

In [ ]:
def ULA_banana(sample, nb_iter, step_size, plot = False):

    sample_size = sample.shape[0]

    dim_var = sample.shape[1]

    for i in range(nb_iter) :

        grad = grad_banana(sample)

        sample -= step_size * grad + np.sqrt(2*step_size) * np.random.randn(sample_size, dim_var) #MOINS Gradient du potentiel
    
    if plot : 

        plot_sample_dx(sample, 'Banana Prior')

    return sample

In [ ]:
sample_init = np.random.randn(1000, 2)

x= ULA_banana(sample_init, 50000, 0.00003)

print(f'Nb NaN {np.sum(np.isnan(x))}')

In [ ]:
grad = np.zeros((1000, 2))

for i in range(1000):

    grad[i] = grad_banana(np.array([[0.1*i, 0.1*i ], [0.1*i, 0.1*i]]))[0] / np.linalg.norm(np.array([0.1*i, 0.1*i]))
    #grad[i] = grad_banana(np.array([[0.1*i, 0.1*i], [0.1*i, 0.1*i]]))[0]

plot_sample_dx(grad)

Therefore, we can use this algo to sample from the prior distribution and generate the observation $y$. 

In [ ]:
true_theta = np.array([-1, 1])

x_star = ULA_banana(np.random.randn(2, 2), 50000, 0.00003)

y_obs = np.dot(true_theta, x_star[0]) + np.random.normal(0, 0.1)

y_obs

Now ULA used to sample from posterior distribution

In [ ]:
def ULA_post_banana(sample, nb_iter, step_size, theta, y_obs, sigma_y):

    sample_size = sample.shape[0]

    dim_var = sample.shape[1]

    for i in tqdm(range(nb_iter)) :

        grad = grad_banana(sample)

        grad += (1/sigma_y**2) * (y_obs - theta[:, np.newaxis].T * sample) * theta

        sample -= step_size * grad + np.sqrt(2*step_size) * np.random.randn(sample_size, dim_var) #MOINS Gradient du potentiel

        if np.sum(np.isnan(sample)) // 2 > 850: 
            return 'Too much NaN'
    
    plot_sample_dx(sample, 'Banana Prior')

    return sample

Instable mais a voir si cela suit la vraie posterior ou pas ? 

In [ ]:
sample_init = np.random.randn(1000, 2)
ULA_post_banana(sample_init, 100000, 0.0000002, true_theta, y_obs, 0.1)

In [ ]:
theta_star_list = np.array([[1, 1], [-1, 1], [3,2], [5,1], [2,0]])

for theta_star in theta_star_list:
    
    y_obs_list = np.zeros(1000)

    for i in tqdm(range(1000)):

        x_star = ULA_banana(np.random.randn(2, 2), 20000, 0.00003)

        y_obs_list[i] = np.dot(theta_star, x_star[0]) + np.random.normal(0, 0.1)

    sns.kdeplot(y_obs_list, fill=True)
    plt.title('Estimation de la Densité par Noyau (KDE)')
    plt.xlabel('Valeur')
    plt.ylabel('Densité')
    plt.show()

